# Data Parsing

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Defining the data file path

In [ ]:
data_file = '../Data/dblp.v12.json'

Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly

In [ ]:
def get_metadata():
    with open(data_file, 'r') as f:
        next(f)                                         #skip the first line
        for line in f:
            yield line
metadata = get_metadata()

In [ ]:
paper_list = []
counter = 0
for paper in metadata:   
    paper = paper.strip(',')       # takes care of the errors with ',' in begining of the line
    if paper == None or paper == '' or paper == '[' or paper == ']' or len(paper) == 0:
        print('I got a null or empty string value for paper in a file')
    else:       
#         print(paper_dict)
#         print("")
#         doc_type = paper_dict.get('doc_type')
        try:
            paper_dict = json.loads(paper)
            year = int(paper_dict.get('year'))
            if year == 2001:
                paper_list.append(paper_dict)
        except:
            pass 
paper_list

In [ ]:
# checking the length of data found
len(paper_list)

In [ ]:
# now add the list into 
papers_df = pd.DataFrame(paper_list)
papers_df.to_csv('../Data/data_paper_sample_Full.csv')

Let's Write the paper dataFrame into pickle for later use

In [ ]:
import pickle
pickle_path = '../Data/papers.pkl'
papers_df.to_pickle(pickle_path)